In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 7)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.36576771736145, Final Batch Loss: 1.0899100303649902
Epoch 2, Loss: 4.349387049674988, Final Batch Loss: 1.0702663660049438
Epoch 3, Loss: 4.3491305112838745, Final Batch Loss: 1.0788629055023193
Epoch 4, Loss: 4.344789981842041, Final Batch Loss: 1.0828319787979126
Epoch 5, Loss: 4.346787810325623, Final Batch Loss: 1.094003438949585
Epoch 6, Loss: 4.330332398414612, Final Batch Loss: 1.0834686756134033
Epoch 7, Loss: 4.307819724082947, Final Batch Loss: 1.0590285062789917
Epoch 8, Loss: 4.302067995071411, Final Batch Loss: 1.0753734111785889
Epoch 9, Loss: 4.2786630392074585, Final Batch Loss: 1.074175477027893
Epoch 10, Loss: 4.240744352340698, Final Batch Loss: 1.0460864305496216
Epoch 11, Loss: 4.18159806728363, Final Batch Loss: 1.0245922803878784
Epoch 12, Loss: 4.144050598144531, Final Batch Loss: 1.033598780632019
Epoch 13, Loss: 4.056296944618225, Final Batch Loss: 1.0162899494171143
Epoch 14, Loss: 3.9756470918655396, Final Batch Loss: 0.9833683967590332
Epo

Epoch 111, Loss: 0.14713040739297867, Final Batch Loss: 0.035206761211156845
Epoch 112, Loss: 0.17698768433183432, Final Batch Loss: 0.07819787412881851
Epoch 113, Loss: 0.12007927522063255, Final Batch Loss: 0.019010182470083237
Epoch 114, Loss: 0.19699268229305744, Final Batch Loss: 0.05125173181295395
Epoch 115, Loss: 0.1240863986313343, Final Batch Loss: 0.0161491259932518
Epoch 116, Loss: 0.14403057098388672, Final Batch Loss: 0.03797454386949539
Epoch 117, Loss: 0.14511089958250523, Final Batch Loss: 0.02481284737586975
Epoch 118, Loss: 0.13968586549162865, Final Batch Loss: 0.03704892843961716
Epoch 119, Loss: 0.1595332771539688, Final Batch Loss: 0.056136008352041245
Epoch 120, Loss: 0.13390304520726204, Final Batch Loss: 0.021936168894171715
Epoch 121, Loss: 0.13044004887342453, Final Batch Loss: 0.05395938456058502
Epoch 122, Loss: 0.20574522577226162, Final Batch Loss: 0.119707390666008
Epoch 123, Loss: 0.11935363803058863, Final Batch Loss: 0.008437984623014927
Epoch 124, L

Epoch 223, Loss: 0.0895784511230886, Final Batch Loss: 0.012076412327587605
Epoch 224, Loss: 0.07614632742479444, Final Batch Loss: 0.03413989022374153
Epoch 225, Loss: 0.09218046255409718, Final Batch Loss: 0.04906642064452171
Epoch 226, Loss: 0.06960407923907042, Final Batch Loss: 0.016022169962525368
Epoch 227, Loss: 0.07487126160413027, Final Batch Loss: 0.030558031052350998
Epoch 228, Loss: 0.09152569901198149, Final Batch Loss: 0.03832918033003807
Epoch 229, Loss: 0.0978116700425744, Final Batch Loss: 0.042822133749723434
Epoch 230, Loss: 0.05182819114997983, Final Batch Loss: 0.004302126355469227
Epoch 231, Loss: 0.09632308967411518, Final Batch Loss: 0.04830231890082359
Epoch 232, Loss: 0.09505121316760778, Final Batch Loss: 0.024045808240771294
Epoch 233, Loss: 0.09059954527765512, Final Batch Loss: 0.010164489038288593
Epoch 234, Loss: 0.057677924167364836, Final Batch Loss: 0.005675687920302153
Epoch 235, Loss: 0.11531026125885546, Final Batch Loss: 0.06804204732179642
Epoch

Epoch 335, Loss: 0.06754338881000876, Final Batch Loss: 0.014920834451913834
Epoch 336, Loss: 0.05854008859023452, Final Batch Loss: 0.022867882624268532
Epoch 337, Loss: 0.07169452076777816, Final Batch Loss: 0.006655833218246698
Epoch 338, Loss: 0.05866815708577633, Final Batch Loss: 0.008490084670484066
Epoch 339, Loss: 0.06398884207010269, Final Batch Loss: 0.008031053468585014
Epoch 340, Loss: 0.053700946271419525, Final Batch Loss: 0.006824522279202938
Epoch 341, Loss: 0.0681737051345408, Final Batch Loss: 0.011279371567070484
Epoch 342, Loss: 0.06830414570868015, Final Batch Loss: 0.007857336662709713
Epoch 343, Loss: 0.08374876203015447, Final Batch Loss: 0.054085370153188705
Epoch 344, Loss: 0.04184260731562972, Final Batch Loss: 0.011018846184015274
Epoch 345, Loss: 0.04890845390036702, Final Batch Loss: 0.0032080146484076977
Epoch 346, Loss: 0.09297464275732636, Final Batch Loss: 0.03734266757965088
Epoch 347, Loss: 0.061701598577201366, Final Batch Loss: 0.00511597655713558

Epoch 448, Loss: 0.02799196122214198, Final Batch Loss: 0.0036357976496219635
Epoch 449, Loss: 0.05080106668174267, Final Batch Loss: 0.0039462074637413025
Epoch 450, Loss: 0.07956235972233117, Final Batch Loss: 0.052728042006492615
Epoch 451, Loss: 0.0623420481570065, Final Batch Loss: 0.021016238257288933
Epoch 452, Loss: 0.04109061602503061, Final Batch Loss: 0.006870650220662355
Epoch 453, Loss: 0.04427370894700289, Final Batch Loss: 0.005129598081111908
Epoch 454, Loss: 0.04846584238111973, Final Batch Loss: 0.02506340481340885
Epoch 455, Loss: 0.04317082790657878, Final Batch Loss: 0.0017700658645480871
Epoch 456, Loss: 0.04321973444893956, Final Batch Loss: 0.009210807271301746
Epoch 457, Loss: 0.060408377554267645, Final Batch Loss: 0.006337106693536043
Epoch 458, Loss: 0.058547146851196885, Final Batch Loss: 0.03747839853167534
Epoch 459, Loss: 0.04784586210735142, Final Batch Loss: 0.004620722029358149
Epoch 460, Loss: 0.04884950316045433, Final Batch Loss: 0.0033113865647464

...

Epoch 4568, Loss: 0.001582436665557907, Final Batch Loss: 0.0001316197303822264
Epoch 4569, Loss: 0.0001760991397077305, Final Batch Loss: 1.6409814634243958e-05
Epoch 4570, Loss: 0.0001459654736208904, Final Batch Loss: 1.7963620848604478e-05
Epoch 4571, Loss: 0.0003640953700596583, Final Batch Loss: 2.3687496650381945e-05
Epoch 4572, Loss: 0.0006515815512102563, Final Batch Loss: 6.765776925021783e-05
Epoch 4573, Loss: 0.0005719371538361884, Final Batch Loss: 0.0005389233119785786
Epoch 4574, Loss: 0.0002444758501951583, Final Batch Loss: 0.00010963201930280775
Epoch 4575, Loss: 0.0007117802952052443, Final Batch Loss: 8.033618541958276e-06
Epoch 4576, Loss: 0.0003502172858134145, Final Batch Loss: 0.00010652669152477756
Epoch 4577, Loss: 0.0001161310692623374, Final Batch Loss: 1.6851379768922925e-05
Epoch 4578, Loss: 0.0021047541598591124, Final Batch Loss: 2.5555909815011546e-06
Epoch 4579, Loss: 0.000825177363367402, Final Batch Loss: 1.0234949513687752e-05
Epoch 4580, Loss: 5.74

Epoch 4669, Loss: 7.655204814227545e-05, Final Batch Loss: 3.482387569420098e-07
Epoch 4670, Loss: 0.0017188554047606885, Final Batch Loss: 0.0016594082117080688
Epoch 4671, Loss: 2.557225320742873e-05, Final Batch Loss: 1.1845600056403782e-06
Epoch 4672, Loss: 0.0007656118120848987, Final Batch Loss: 2.875653990486171e-05
Epoch 4673, Loss: 0.0004813518382889015, Final Batch Loss: 2.7885371309821494e-06
Epoch 4674, Loss: 0.00010114249380421825, Final Batch Loss: 1.195775439555291e-06
Epoch 4675, Loss: 0.00022760849651604076, Final Batch Loss: 0.00013193563790991902
Epoch 4676, Loss: 4.305782638880373e-05, Final Batch Loss: 4.160408934694715e-06
Epoch 4677, Loss: 0.00011218858651318442, Final Batch Loss: 9.265132575819734e-06
Epoch 4678, Loss: 4.8280058194904996e-05, Final Batch Loss: 3.677824861370027e-05
Epoch 4679, Loss: 6.264105195441516e-05, Final Batch Loss: 3.137055318802595e-05
Epoch 4680, Loss: 0.0004137520018048235, Final Batch Loss: 0.00040026966598816216
Epoch 4681, Loss: 7.

Epoch 4775, Loss: 0.0001726123296066362, Final Batch Loss: 6.6643829086387996e-06
Epoch 4776, Loss: 0.0001684342041698983, Final Batch Loss: 5.836717173224315e-05
Epoch 4777, Loss: 0.002548639185988577, Final Batch Loss: 1.7041682440321892e-05
Epoch 4778, Loss: 0.003986247045077107, Final Batch Loss: 0.0034687663428485394
Epoch 4779, Loss: 0.0002034736648965918, Final Batch Loss: 9.573785064276308e-05
Epoch 4780, Loss: 8.721433914615773e-05, Final Batch Loss: 1.1551746865734458e-05
Epoch 4781, Loss: 7.667553450119158e-05, Final Batch Loss: 2.5500546598777873e-06
Epoch 4782, Loss: 0.00020155291304035927, Final Batch Loss: 5.401147518568905e-06
Epoch 4783, Loss: 0.0003672337752504973, Final Batch Loss: 0.00012784886348526925
Epoch 4784, Loss: 0.13328667181122, Final Batch Loss: 0.1323057860136032
Epoch 4785, Loss: 0.0008107808098429814, Final Batch Loss: 2.9995855584274977e-05
Epoch 4786, Loss: 0.0007171868774094037, Final Batch Loss: 0.0003034147375728935
Epoch 4787, Loss: 0.03236017573

Epoch 4883, Loss: 5.656746702698001e-05, Final Batch Loss: 5.36825700692134e-06
Epoch 4884, Loss: 5.1376810461079e-05, Final Batch Loss: 1.650564809096977e-05
Epoch 4885, Loss: 5.454553502204362e-05, Final Batch Loss: 1.8976550563820638e-05
Epoch 4886, Loss: 9.528966006655537e-05, Final Batch Loss: 1.2165988664492033e-05
Epoch 4887, Loss: 0.0002673004910320742, Final Batch Loss: 1.7876040146802552e-05
Epoch 4888, Loss: 0.00013095297981635667, Final Batch Loss: 4.1351362597197294e-05
Epoch 4889, Loss: 0.0026204435380350333, Final Batch Loss: 1.8618206013343297e-05
Epoch 4890, Loss: 0.004752474219458236, Final Batch Loss: 0.0003209979913663119
Epoch 4891, Loss: 0.000765075952585903, Final Batch Loss: 0.0006522221374325454
Epoch 4892, Loss: 0.00021656762692146003, Final Batch Loss: 4.456871829461306e-05
Epoch 4893, Loss: 0.00020620104487534263, Final Batch Loss: 3.731018750841031e-06
Epoch 4894, Loss: 0.003006481816555606, Final Batch Loss: 0.0016936881002038717
Epoch 4895, Loss: 0.000142

Epoch 4994, Loss: 0.00014373666704159405, Final Batch Loss: 3.2134637422132073e-06
Epoch 4995, Loss: 0.00016183656589419115, Final Batch Loss: 2.3274871637113392e-05
Epoch 4996, Loss: 0.0025961208029912086, Final Batch Loss: 6.535270949825644e-05
Epoch 4997, Loss: 0.001687968860210276, Final Batch Loss: 0.0014792144065722823
Epoch 4998, Loss: 0.00010432118506287225, Final Batch Loss: 6.915877020219341e-05
Epoch 4999, Loss: 0.00023480498566641472, Final Batch Loss: 6.011160621710587e-06
Epoch 5000, Loss: 0.00014946948476790567, Final Batch Loss: 4.522071776591474e-06


In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[83  3  1]
 [ 0 63  0]
 [ 0  0 70]]
              precision    recall  f1-score   support

           0    1.00000   0.95402   0.97647        87
           1    0.95455   1.00000   0.97674        63
           2    0.98592   1.00000   0.99291        70

    accuracy                        0.98182       220
   macro avg    0.98015   0.98467   0.98204       220
weighted avg    0.98250   0.98182   0.98178       220



In [11]:
n_samples = 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U6A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples * 7)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_13 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U6A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_14 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples * 7)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_15 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_16 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_17 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_18 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_19 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U5A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_20 = gen(latent_vectors).detach().numpy()

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U6A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_21 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples * 7) + 1

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12,
                               fake_features_13, fake_features_14, fake_features_15, fake_features_16, fake_features_17, fake_features_18,
                               fake_features_19, fake_features_20, fake_features_21))
fake_labels = np.concatenate((y_1, y_2, y_3))

fake_features = torch.Tensor(fake_features)

In [12]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[68  2  0]
 [ 0 70  0]
 [ 0  0 70]]
              precision    recall  f1-score   support

         0.0    1.00000   0.97143   0.98551        70
         1.0    0.97222   1.00000   0.98592        70
         2.0    1.00000   1.00000   1.00000        70

    accuracy                        0.99048       210
   macro avg    0.99074   0.99048   0.99047       210
weighted avg    0.99074   0.99048   0.99047       210



# User Classifier

In [13]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11, 14]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [14]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    elif y[k] == 7:
        y[k] = 3
    elif y[k] == 8:
        y[k] = 4
    elif y[k] == 11:
        y[k] = 5
    else:
        y[k] = 6

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [16]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.829120397567749, Final Batch Loss: 1.9629391431808472
Epoch 2, Loss: 7.806324481964111, Final Batch Loss: 1.9320296049118042
Epoch 3, Loss: 7.799121022224426, Final Batch Loss: 1.9395220279693604
Epoch 4, Loss: 7.78170371055603, Final Batch Loss: 1.9449036121368408
Epoch 5, Loss: 7.778874039649963, Final Batch Loss: 1.9326989650726318
Epoch 6, Loss: 7.762550473213196, Final Batch Loss: 1.9288957118988037
Epoch 7, Loss: 7.769577980041504, Final Batch Loss: 1.9299522638320923
Epoch 8, Loss: 7.76414680480957, Final Batch Loss: 1.9429709911346436
Epoch 9, Loss: 7.751281023025513, Final Batch Loss: 1.9246987104415894
Epoch 10, Loss: 7.754444599151611, Final Batch Loss: 1.9341353178024292
Epoch 11, Loss: 7.73308002948761, Final Batch Loss: 1.9240518808364868
Epoch 12, Loss: 7.729907155036926, Final Batch Loss: 1.9358093738555908
Epoch 13, Loss: 7.705235481262207, Final Batch Loss: 1.9148224592208862
Epoch 14, Loss: 7.7063000202178955, Final Batch Loss: 1.945185661315918
Epoc

Epoch 115, Loss: 4.072989761829376, Final Batch Loss: 1.0495809316635132
Epoch 116, Loss: 4.043751001358032, Final Batch Loss: 1.066024899482727
Epoch 117, Loss: 4.034191310405731, Final Batch Loss: 0.9591153264045715
Epoch 118, Loss: 4.095159947872162, Final Batch Loss: 1.1093199253082275
Epoch 119, Loss: 4.036996603012085, Final Batch Loss: 0.9527297616004944
Epoch 120, Loss: 4.078827440738678, Final Batch Loss: 1.0388680696487427
Epoch 121, Loss: 3.941320776939392, Final Batch Loss: 1.065710425376892
Epoch 122, Loss: 4.038117349147797, Final Batch Loss: 1.0025553703308105
Epoch 123, Loss: 4.146547794342041, Final Batch Loss: 1.125451922416687
Epoch 124, Loss: 3.916926145553589, Final Batch Loss: 0.8838779926300049
Epoch 125, Loss: 3.977644979953766, Final Batch Loss: 0.9953639507293701
Epoch 126, Loss: 3.8860031962394714, Final Batch Loss: 0.8882778882980347
Epoch 127, Loss: 3.903688609600067, Final Batch Loss: 0.8915362358093262
Epoch 128, Loss: 3.900764286518097, Final Batch Loss:

Epoch 231, Loss: 3.002963602542877, Final Batch Loss: 0.8006185293197632
Epoch 232, Loss: 2.7694727778434753, Final Batch Loss: 0.7013657093048096
Epoch 233, Loss: 2.9162025451660156, Final Batch Loss: 0.6161056160926819
Epoch 234, Loss: 2.9618998765945435, Final Batch Loss: 0.8408477902412415
Epoch 235, Loss: 2.9469281435012817, Final Batch Loss: 0.7802278399467468
Epoch 236, Loss: 2.7670750617980957, Final Batch Loss: 0.6542277336120605
Epoch 237, Loss: 2.920811176300049, Final Batch Loss: 0.6909490823745728
Epoch 238, Loss: 2.9888681173324585, Final Batch Loss: 0.7519965767860413
Epoch 239, Loss: 2.7348233461380005, Final Batch Loss: 0.7362764477729797
Epoch 240, Loss: 2.9096741676330566, Final Batch Loss: 0.7161902189254761
Epoch 241, Loss: 2.8940380215644836, Final Batch Loss: 0.6571955680847168
Epoch 242, Loss: 2.8276575803756714, Final Batch Loss: 0.7089771628379822
Epoch 243, Loss: 2.7883923053741455, Final Batch Loss: 0.6893912553787231
Epoch 244, Loss: 3.010918080806732, Fina

Epoch 347, Loss: 2.458435356616974, Final Batch Loss: 0.6047745943069458
Epoch 348, Loss: 2.4628159403800964, Final Batch Loss: 0.5999388694763184
Epoch 349, Loss: 2.4457753896713257, Final Batch Loss: 0.5753253102302551
Epoch 350, Loss: 2.5367594361305237, Final Batch Loss: 0.6192055344581604
Epoch 351, Loss: 2.4964385628700256, Final Batch Loss: 0.5948818325996399
Epoch 352, Loss: 2.527197480201721, Final Batch Loss: 0.6169732809066772
Epoch 353, Loss: 2.491900324821472, Final Batch Loss: 0.7192424535751343
Epoch 354, Loss: 2.572737395763397, Final Batch Loss: 0.6426435708999634
Epoch 355, Loss: 2.4468822479248047, Final Batch Loss: 0.57635498046875
Epoch 356, Loss: 2.3472448587417603, Final Batch Loss: 0.6551461815834045
Epoch 357, Loss: 2.519586205482483, Final Batch Loss: 0.6201153993606567
Epoch 358, Loss: 2.409421384334564, Final Batch Loss: 0.6453325152397156
Epoch 359, Loss: 2.4960864186286926, Final Batch Loss: 0.6171118021011353
Epoch 360, Loss: 2.4876797199249268, Final Bat

Epoch 469, Loss: 2.307913064956665, Final Batch Loss: 0.5811082720756531
Epoch 470, Loss: 2.157164990901947, Final Batch Loss: 0.5467043519020081
Epoch 471, Loss: 2.2013357877731323, Final Batch Loss: 0.5021005868911743
Epoch 472, Loss: 2.265878677368164, Final Batch Loss: 0.5734277963638306
Epoch 473, Loss: 2.2826411724090576, Final Batch Loss: 0.6142022609710693
Epoch 474, Loss: 2.185527205467224, Final Batch Loss: 0.526756763458252
Epoch 475, Loss: 2.393193006515503, Final Batch Loss: 0.7754323482513428
Epoch 476, Loss: 2.267684817314148, Final Batch Loss: 0.6578729748725891
Epoch 477, Loss: 2.2593387961387634, Final Batch Loss: 0.5988275408744812
Epoch 478, Loss: 2.1560438871383667, Final Batch Loss: 0.46074652671813965
Epoch 479, Loss: 2.302128553390503, Final Batch Loss: 0.540520191192627
Epoch 480, Loss: 2.177589327096939, Final Batch Loss: 0.48236533999443054
Epoch 481, Loss: 2.2503591179847717, Final Batch Loss: 0.5488118529319763
Epoch 482, Loss: 2.2300037145614624, Final Bat

...

Epoch 4461, Loss: 0.9198978692293167, Final Batch Loss: 0.2627718448638916
Epoch 4462, Loss: 0.9411504566669464, Final Batch Loss: 0.2656466066837311
Epoch 4463, Loss: 0.9544029533863068, Final Batch Loss: 0.22422342002391815
Epoch 4464, Loss: 1.023260846734047, Final Batch Loss: 0.239740788936615
Epoch 4465, Loss: 0.9649959057569504, Final Batch Loss: 0.2620237469673157
Epoch 4466, Loss: 1.0557481795549393, Final Batch Loss: 0.27951183915138245
Epoch 4467, Loss: 0.9061347246170044, Final Batch Loss: 0.26471421122550964
Epoch 4468, Loss: 0.9815859645605087, Final Batch Loss: 0.21729245781898499
Epoch 4469, Loss: 1.087326243519783, Final Batch Loss: 0.2307257205247879
Epoch 4470, Loss: 0.9766454696655273, Final Batch Loss: 0.20744003355503082
Epoch 4471, Loss: 1.0343541502952576, Final Batch Loss: 0.2528461217880249
Epoch 4472, Loss: 1.0736842602491379, Final Batch Loss: 0.2332950085401535
Epoch 4473, Loss: 0.9690720736980438, Final Batch Loss: 0.22153174877166748
Epoch 4474, Loss: 0.97

Epoch 4569, Loss: 1.0850637257099152, Final Batch Loss: 0.17979195713996887
Epoch 4570, Loss: 0.9096919149160385, Final Batch Loss: 0.2131901979446411
Epoch 4571, Loss: 1.0218788385391235, Final Batch Loss: 0.20143365859985352
Epoch 4572, Loss: 0.9190466105937958, Final Batch Loss: 0.28285548090934753
Epoch 4573, Loss: 0.7977585941553116, Final Batch Loss: 0.17601478099822998
Epoch 4574, Loss: 0.9776347726583481, Final Batch Loss: 0.2853783369064331
Epoch 4575, Loss: 0.8507059812545776, Final Batch Loss: 0.18016308546066284
Epoch 4576, Loss: 0.9820450097322464, Final Batch Loss: 0.1654045730829239
Epoch 4577, Loss: 1.10960054397583, Final Batch Loss: 0.2969166934490204
Epoch 4578, Loss: 0.9253563433885574, Final Batch Loss: 0.2517324388027191
Epoch 4579, Loss: 1.0076048076152802, Final Batch Loss: 0.19598506391048431
Epoch 4580, Loss: 0.9059172123670578, Final Batch Loss: 0.2832389771938324
Epoch 4581, Loss: 0.9178643375635147, Final Batch Loss: 0.1797615885734558
Epoch 4582, Loss: 1.0

Epoch 4681, Loss: 0.9710310697555542, Final Batch Loss: 0.22493164241313934
Epoch 4682, Loss: 0.9161009937524796, Final Batch Loss: 0.2572633922100067
Epoch 4683, Loss: 0.9982617795467377, Final Batch Loss: 0.26724639534950256
Epoch 4684, Loss: 0.9847741574048996, Final Batch Loss: 0.16889674961566925
Epoch 4685, Loss: 1.1034162044525146, Final Batch Loss: 0.13707862794399261
Epoch 4686, Loss: 0.961030513048172, Final Batch Loss: 0.27303630113601685
Epoch 4687, Loss: 0.9084748476743698, Final Batch Loss: 0.24312934279441833
Epoch 4688, Loss: 0.9229920655488968, Final Batch Loss: 0.2489493489265442
Epoch 4689, Loss: 0.9777169227600098, Final Batch Loss: 0.29107582569122314
Epoch 4690, Loss: 1.022699236869812, Final Batch Loss: 0.27206552028656006
Epoch 4691, Loss: 0.8564888089895248, Final Batch Loss: 0.19914217293262482
Epoch 4692, Loss: 0.9932166337966919, Final Batch Loss: 0.20506072044372559
Epoch 4693, Loss: 0.9552560299634933, Final Batch Loss: 0.1893412172794342
Epoch 4694, Loss:

Epoch 4793, Loss: 0.9145811498165131, Final Batch Loss: 0.27652856707572937
Epoch 4794, Loss: 0.9974191784858704, Final Batch Loss: 0.2418251484632492
Epoch 4795, Loss: 0.9934609234333038, Final Batch Loss: 0.2053476721048355
Epoch 4796, Loss: 1.0912388563156128, Final Batch Loss: 0.23391297459602356
Epoch 4797, Loss: 1.101365089416504, Final Batch Loss: 0.2705492377281189
Epoch 4798, Loss: 1.028943806886673, Final Batch Loss: 0.34359312057495117
Epoch 4799, Loss: 1.0073984861373901, Final Batch Loss: 0.25200146436691284
Epoch 4800, Loss: 0.8570845127105713, Final Batch Loss: 0.12896399199962616
Epoch 4801, Loss: 1.049721121788025, Final Batch Loss: 0.28839626908302307
Epoch 4802, Loss: 0.9110043942928314, Final Batch Loss: 0.21841445565223694
Epoch 4803, Loss: 0.9383976012468338, Final Batch Loss: 0.16618885099887848
Epoch 4804, Loss: 1.007545605301857, Final Batch Loss: 0.23693132400512695
Epoch 4805, Loss: 1.0063494592905045, Final Batch Loss: 0.2088492065668106
Epoch 4806, Loss: 0.

Epoch 4907, Loss: 0.9518777877092361, Final Batch Loss: 0.31831297278404236
Epoch 4908, Loss: 0.9273402988910675, Final Batch Loss: 0.25718656182289124
Epoch 4909, Loss: 0.8750833570957184, Final Batch Loss: 0.23946848511695862
Epoch 4910, Loss: 0.9917814284563065, Final Batch Loss: 0.21335434913635254
Epoch 4911, Loss: 0.968689814209938, Final Batch Loss: 0.2629547119140625
Epoch 4912, Loss: 1.048228606581688, Final Batch Loss: 0.2739087641239166
Epoch 4913, Loss: 0.9948750138282776, Final Batch Loss: 0.26643067598342896
Epoch 4914, Loss: 0.9924626499414444, Final Batch Loss: 0.17496155202388763
Epoch 4915, Loss: 0.9443012177944183, Final Batch Loss: 0.2562863826751709
Epoch 4916, Loss: 0.9876133501529694, Final Batch Loss: 0.26670366525650024
Epoch 4917, Loss: 1.0746541917324066, Final Batch Loss: 0.2245214879512787
Epoch 4918, Loss: 0.987550288438797, Final Batch Loss: 0.2409908026456833
Epoch 4919, Loss: 0.9435631632804871, Final Batch Loss: 0.26446813344955444
Epoch 4920, Loss: 0.

In [17]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[44  0  0  0  0  0  0]
 [ 0 24  0  1  4  2  1]
 [ 0  0 23  0  0  0  0]
 [ 0  0  0 36  0  0  1]
 [ 1  1  0  0 22  0  0]
 [ 1  0  0  0  1 32  0]
 [ 0  0  0  2  0  0 24]]
              precision    recall  f1-score   support

           0    0.95652   1.00000   0.97778        44
           1    0.96000   0.75000   0.84211        32
           2    1.00000   1.00000   1.00000        23
           3    0.92308   0.97297   0.94737        37
           4    0.81481   0.91667   0.86275        24
           5    0.94118   0.94118   0.94118        34
           6    0.92308   0.92308   0.92308        26

    accuracy                        0.93182       220
   macro avg    0.93124   0.92913   0.92775       220
weighted avg    0.93417   0.93182   0.93058       220



In [18]:
fake_labels = [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [5] * n_samples + [6] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [5] * n_samples + [6] * n_samples + [0] * n_samples + [1] * n_samples + [2] * n_samples + [3] * n_samples + [4] * n_samples + [5] * n_samples + [6] * n_samples
fake_labels = np.asarray(fake_labels)

In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(fake_labels, preds.cpu()))
print(metrics.classification_report(fake_labels, preds.cpu(), digits = 5))

[[21  0  2  0  3  1  3]
 [ 2 18  2  4  2  1  1]
 [ 0  0 18 11  1  0  0]
 [ 0  5  0 20  4  0  1]
 [ 3  1  2  1 20  3  0]
 [ 6  0  8  0  0 16  0]
 [ 0  0  0  0  0  2 28]]
              precision    recall  f1-score   support

           0    0.65625   0.70000   0.67742        30
           1    0.75000   0.60000   0.66667        30
           2    0.56250   0.60000   0.58065        30
           3    0.55556   0.66667   0.60606        30
           4    0.66667   0.66667   0.66667        30
           5    0.69565   0.53333   0.60377        30
           6    0.84848   0.93333   0.88889        30

    accuracy                        0.67143       210
   macro avg    0.67644   0.67143   0.67002       210
weighted avg    0.67644   0.67143   0.67002       210

